In [1]:
!pip install -q wavencoder

In [2]:
RATE = '8000' 

CATES = ['female', 'male']

BASE_ORIGINAL_TRAIN = '../input/gendersclassification-vdt-2022/dataset_v2/train'

BASE_TRAIN = BASE_ORIGINAL_TRAIN

BASE_PUBLIC_TEST = '../input/gendersclassification-vdt-2022/public-test/public-test/wav'
BASE_PRIVATE_TEST = '../input/gendersclassification-vdt-2022/private-test/private-test/wav'
NUM_WORKERS = 4
import torch
INFER_ONLY = True # change this to False to train the model again 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# wavencoder.utils.download_noise_dataset('noise_dataset', download_all=False, noise_envs=['kitchen', 'metro'])

In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import librosa
def get_fn_lbs():
    lbs = []
    fns = []
    for name_dir in os.listdir(BASE_TRAIN):
        for i in os.listdir(os.path.join(BASE_TRAIN,name_dir)):
            if name_dir == 'male':
                for j in os.listdir(os.path.join(BASE_TRAIN,name_dir,i)):
                    dur = librosa.get_duration(filename=os.path.join(BASE_TRAIN,name_dir,i,j))
                    if dur >1:
                        lbs.append(1)
                        fns.append(os.path.join(BASE_TRAIN,name_dir,i,j))
            if name_dir == 'female':
                for j in os.listdir(os.path.join(BASE_TRAIN,name_dir,i)):
                    dur = librosa.get_duration(filename=os.path.join(BASE_TRAIN,name_dir,i,j))
                    if dur >1:
                        lbs.append(0)
                        fns.append(os.path.join(BASE_TRAIN,name_dir,i,j))
    return lbs,fns

lbs,fns= get_fn_lbs()
    
train_fns, val_fns, train_lbs, val_lbs = train_test_split(fns, lbs, test_size=0.1, random_state=42,shuffle = True)
def get_fn_test():
    lbs = []
    fns = []
    df = pd.read_csv("../input/gendersclassification-vdt-2022/public-test/public-test/test_files.txt", header=None,names=["gender", "filename"])
    for index, row in df.iterrows():
        if str(row['gender']) == 'M':
            dur = librosa.get_duration(filename=os.path.join('../input/gendersclassification-vdt-2022/public-test/public-test/wav',str(row['filename'])))
            if dur >1:
                lbs.append(1)
                fns.append(os.path.join('../input/gendersclassification-vdt-2022/public-test/public-test/wav',str(row['filename'])))
        elif str(row['gender']) == 'F':
            dur = librosa.get_duration(filename=os.path.join('../input/gendersclassification-vdt-2022/public-test/public-test/wav',str(row['filename'])))
            if dur >1:
                lbs.append(0)
                fns.append(os.path.join('../input/gendersclassification-vdt-2022/public-test/public-test/wav',str(row['filename'])))
    return lbs,fns
test_lbs,test_fns = get_fn_test()
def get_fn_submit():
    fns = []
    lbs = []
    for i in os.listdir('../input/gendersclassification-vdt-2022/private-test/private-test/wav'):
        fns.append(os.path.join('../input/gendersclassification-vdt-2022/private-test/private-test/wav',i))
        lbs.append(0)
    return lbs,fns
submit_lbs,submit_fns = get_fn_submit()

In [7]:
from torch.utils.data import Dataset
import torchaudio



class Dataset(Dataset):
    def __init__(self, filenames, labels, hparams, is_train=True,):
        
        self.files = filenames
        self.lbs = labels
        self.wav_len = hparams.timit_wav_len
        self.data_type = hparams.data_type
        self.is_train = is_train
        self.noise_dataset_path='noise_dataset'
        
#         if self.noise_dataset_path:
#             self.train_transform = wavencoder.transforms.Compose([
#                 wavencoder.transforms.PadCrop(pad_crop_length=self.wav_len, pad_position='random', crop_position='random'),
#                 wavencoder.transforms.AdditiveNoise(self.noise_dataset_path, p=0.5),
#                 wavencoder.transforms.Clipping(p=0.5),
#                 ])
#         else:
#             self.train_transform = wavencoder.transforms.Compose([
#                 wavencoder.transforms.PadCrop(pad_crop_length=self.wav_len, pad_position='center', crop_position='random'),
#                 wavencoder.transforms.Clipping(p=0.5),
#                 ])


        self.train_transform = wavencoder.transforms.Compose([
                wavencoder.transforms.PadCrop(pad_crop_length=self.wav_len, pad_position='center', crop_position='random'),
                wavencoder.transforms.Clipping(p=0.5),
                ])

        self.test_transform = wavencoder.transforms.Compose([
            wavencoder.transforms.PadCrop(pad_crop_length=self.wav_len, pad_position='left', crop_position='center')
            ])

        if self.data_type == 'spectral':
            # self.spectral_transform = torchaudio.transforms.MelSpectrogram(normalized=True)
            self.spectral_transform = torchaudio.transforms.MFCC(n_mfcc=40,sample_rate=8000, log_mels=True)
            self.spec_aug = wavencoder.transforms.Compose([  
                torchaudio.transforms.FrequencyMasking(5),
                torchaudio.transforms.TimeMasking(5),
            ])

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        file = self.files[idx]
        gender = self.lbs[idx]
        wav, _ = torchaudio.load(file)
        if self.is_train:
            wav = self.train_transform(wav)  
            if self.data_type == 'spectral':
                wav = self.spectral_transform(wav)
                


        else:
            if self.data_type == 'spectral':
                wav = self.spectral_transform(wav)
        
        return wav, gender

In [8]:

import torch.nn as nn


class Wav2VecLSTM(nn.Module):
    def __init__(self, lstm_h, lstm_inp=512):
        super().__init__()
        self.encoder = wavencoder.models.Wav2Vec(pretrained=True)
        for param in self.encoder.parameters():
            param.requires_grad = False

        for param in self.encoder.feature_extractor.conv_layers[5:].parameters():
            param.requires_grad = True
        
        self.lstm = nn.LSTM(lstm_inp, lstm_h, batch_first=True)
        self.attention = wavencoder.layers.SoftAttention(lstm_h, lstm_h)
        self.gender_classifier = nn.Sequential(
            nn.Linear(lstm_h, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        output, (hidden, _) = self.lstm(x.transpose(1,2))
        attn_output = self.attention(output)
        gender = self.gender_classifier(attn_output)
        return gender



class SpectralLSTM(nn.Module):
    def __init__(self, lstm_h):
        super().__init__()

        self.lstm = nn.LSTM(128, lstm_h, batch_first=True)
        self.attention = wavencoder.layers.SoftAttention(lstm_h, lstm_h)
        
        self.gender_classifier = nn.Sequential(
            nn.Linear(lstm_h, lstm_h),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(lstm_h, 1),
            nn.Sigmoid())

    def forward(self, x):
        x = x.squeeze(1)
        output, (hidden, _) = self.lstm(x.transpose(1,2))
        attn_output = self.attention(output)

        gender = self.gender_classifier(attn_output)
        return gender

class SpectralCNNLSTM(nn.Module):
    def __init__(self, lstm_h):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(40, int(lstm_h/2), 5),
            nn.ReLU(),
            nn.BatchNorm1d(int(lstm_h/2)),
            nn.Conv1d(int(lstm_h/2), int(lstm_h/2), 5),
            nn.ReLU(),
            nn.BatchNorm1d(int(lstm_h/2)),
        )
        self.lstm = nn.LSTM(int(lstm_h/2), int(lstm_h/2), batch_first=True)
        self.attention = wavencoder.layers.SoftAttention(int(lstm_h/2), int(lstm_h/2))
    
        self.gender_classifier = nn.Sequential(
            nn.Linear(int(lstm_h/2), int(lstm_h/2)),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(int(lstm_h/2), 1),
            nn.Sigmoid())

    def forward(self, x):
        x = x.squeeze(1)
        x = self.encoder(x)
        output, (hidden, _) = self.lstm(x.transpose(1,2))
        attn_output = self.attention(output)

        gender = self.gender_classifier(attn_output)
        return gender






In [9]:
!pip install -q torch_optimizer

In [10]:

import torch.nn as nn


import pytorch_lightning as pl

from torchmetrics import MeanSquaredError  as MSE
from torchmetrics import Accuracy

import wavencoder
import torch_optimizer as optim






class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.CrossEntropyLoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

class LightningModel(pl.LightningModule):
    def __init__(self, HPARAMS):
        super().__init__()

        self.save_hyperparameters()
        self.models = {
            'wav2vecLSTMAttn': Wav2VecLSTM,
            'spectralCNNLSTM' : SpectralCNNLSTM,
            'MultiScale' : SpectralMultiScale,
            'SpectralLSTM': SpectralLSTM,
            

        }
        self.model = self.models[HPARAMS['model_type']](HPARAMS['hidden_size'])

        self.classification_criterion = MSE()
        self.accuracy = Accuracy()


        self.lr = HPARAMS['lr']


        print(f"Model Details: #Params = {self.count_total_parameters()}\t#Trainable Params = {self.count_trainable_parameters()}")

    def count_total_parameters(self):
            return sum(p.numel() for p in self.parameters())
    
    def count_trainable_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = optim.DiffGrad(self.parameters(), lr=self.lr)
        return [optimizer]

    def training_step(self, batch, batch_idx):
        x, y_g = batch
        y_hat_g = self(x)
        y_g = y_g.view(-1).float()
        y_hat_g = y_hat_g.view(-1).float()

        gender_loss = self.classification_criterion(y_hat_g, y_g)
        loss =  gender_loss

        
        gender_acc = self.accuracy((y_hat_g>0.5).long(), y_g.long())


        return {'loss':loss, 
                'train_gender_acc':gender_acc,
                }
    
    def training_epoch_end(self, outputs):
        n_batch = len(outputs)
        loss = torch.tensor([x['loss'] for x in outputs]).mean()
        gender_acc = torch.tensor([x['train_gender_acc'] for x in outputs]).mean()

        self.log('train/loss' , loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('train/g',gender_acc, on_step=False, on_epoch=True, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        x, y_g = batch
        y_hat_g = self(x)
        y_g =  y_g.view(-1).float()
        y_hat_g = y_hat_g.view(-1).float()

        gender_loss = self.classification_criterion(y_hat_g, y_g)
        loss = self.gamma * gender_loss

        gender_acc = self.accuracy((y_hat_g>0.5).long(), y_g.long())

        return {'val_loss':loss, 
                'val_gender_acc':gender_acc}

    def validation_epoch_end(self, outputs):
        n_batch = len(outputs)
        val_loss = torch.tensor([x['val_loss'] for x in outputs]).mean()
        gender_acc = torch.tensor([x['val_gender_acc'] for x in outputs]).mean()
        self.log('val/loss' , val_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val/g',gender_acc, on_step=False, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y_g = batch
        y_hat_g = self(x)
        y_g =  y_g.view(-1).float()
        y_hat_g = y_hat_g.view(-1).float()

        gender_acc = self.accuracy((y_hat_g>0.5).long(), y_g.long())

        idx = y_g.view(-1).long()
        female_idx = torch.nonzero(idx).view(-1)
        male_idx = torch.nonzero(1-idx).view(-1)

       

        return {
                'test_gender_acc':gender_acc}

    def test_epoch_end(self, outputs):
        n_batch = len(outputs)
        
        gender_acc = torch.tensor([x['test_gender_acc'] for x in outputs]).mean()

        pbar = {
                'test_gender_acc':gender_acc.item()}
        self.logger.log_hyperparams(pbar)
        self.log_dict(pbar)

In [11]:
import os

class Config(object):

    
    # length of wav files for training and testing
    wav_len = 2 * 8000
    # 16000 * 2

    batch_size = 1024
    epochs = 150
    
    # loss = alpha * height_loss + beta * age_loss + gamma * gender_loss
    alpha = 1
    beta = 1
    gamma = 1

    # training type - AHG/H
    training_type = 'AHG'

    # data type - raw/spectral
    data_type = 'spectral' 

    # model type
    ## AHG 
    # wav2vecLSTMAttn/spectralCNNLSTM/MultiScale,SpectralLSTM
    model_type = 'spectralCNNLSTM'
    
    # hidden dimension of LSTM and Dense Layers
    hidden_size = 128

    # No of GPUs for training and no of workers for datalaoders
    gpu = '-1'
    n_workers = 2

    # model checkpoint to continue from
    model_checkpoint = None
    
    # noise dataset for augmentation
    noise_dataset_path = 'noise_dataset'

    # LR of optimizer
    lr = 1e-4

    run_name = data_type + '_' + training_type + '_' + model_type


In [12]:

from argparse import ArgumentParser

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer


import torch
import torch.utils.data as data



# SEED
SEED=100
pl.utilities.seed.seed_everything(SEED)
torch.manual_seed(SEED)


if __name__ == "__main__":

    parser = ArgumentParser(add_help=True)
    parser.add_argument('--wav_len', type=int, default=Config.wav_len)
    parser.add_argument('--batch_size', type=int, default=Config.batch_size)
    parser.add_argument('--epochs', type=int, default=Config.epochs)
    parser.add_argument('--alpha', type=float, default=Config.alpha)
    parser.add_argument('--beta', type=float, default=Config.beta)
    parser.add_argument('--gamma', type=float, default=Config.gamma)
    parser.add_argument('--hidden_size', type=float, default=Config.hidden_size)
    parser.add_argument('--lr', type=float, default=Config.lr)
    parser.add_argument('--gpu', type=int, default=Config.gpu)
    parser.add_argument('--n_workers', type=int, default=Config.n_workers)
    parser.add_argument('--dev', type=str, default=False)
    parser.add_argument('--model_checkpoint', type=str, default=Config.model_checkpoint)
    parser.add_argument('--noise_dataset_path', type=str, default=Config.noise_dataset_path)
    parser.add_argument('--model_type', type=str, default=Config.model_type)
    parser.add_argument('--training_type', type=str, default=Config.training_type)
    parser.add_argument('--data_type', type=str, default=Config.data_type)

    parser = pl.Trainer.add_argparse_args(parser)
    hparams, unknown = parser.parse_known_args()
    print(f'Training Model on TIMIT Dataset\n#Cores = {hparams.n_workers}\t#GPU = {hparams.gpu}')

    # Training, Validation and Testing Dataset
    ## Training Dataset
    train_set = Dataset(filenames = train_fns, labels = train_lbs ,
        hparams = hparams
    )
    ## Training DataLoader
    trainloader = data.DataLoader(
        train_set, 
        batch_size=hparams.batch_size, 
        shuffle=True, 
        num_workers=hparams.n_workers
    )
    ## Validation Dataset
    valid_set = Dataset(
        filenames =val_fns,labels = val_lbs,
        hparams = hparams,
        is_train=True
    )
    ## Validation Dataloader
    valloader = data.DataLoader(
        valid_set, 
        batch_size=
        hparams.batch_size, 
        shuffle=False, 
        num_workers=hparams.n_workers
    )
    ## Testing Dataset
    test_set = Dataset(
        filenames = test_fns, labels = test_lbs ,
        hparams = hparams,
        is_train=True
    )
    ## Testing Dataloader
    testloader = data.DataLoader(
        test_set, 
        batch_size=hparams.batch_size, 
        # hparams.batch_size, 
        shuffle=False, 
        num_workers=hparams.n_workers
    )
    

    print('Dataset Split (Train, Validation, Test)=', len(train_set), len(valid_set), len(test_set))

    model = LightningModel(vars(hparams))

    checkpoint_callback = ModelCheckpoint(
        monitor='val/loss', 
        mode='min',
        verbose=1)

    trainer = Trainer(
        fast_dev_run=hparams.dev, 
        gpus=hparams.gpu, 
        max_epochs=hparams.epochs, 
        checkpoint_callback=checkpoint_callback,
        callbacks=[
            EarlyStopping(
                monitor='val/loss',
                min_delta=0.00,
                patience=20,
                verbose=True,
                mode='min'
                )
        ],
        resume_from_checkpoint=hparams.model_checkpoint,
        )

    trainer.fit(model, trainloader, valloader)



Training Model on TIMIT Dataset
#Cores = 2	#GPU = -1
Dataset Split (Train, Validation, Test)= 13072 1453 2868
Model Details: #Params = 75393	#Trainable Params = 75393


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7fd0a273a950>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7fd0a273a950>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1931: PossibleUserWarning: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [14]:

submit_set = Dataset(
    filenames = submit_fns, labels = submit_lbs ,
    hparams = hparams,
    is_train=True
)
## Testing Dataloader
submitloader = data.DataLoader(
    submit_set,
    batch_size=hparams.batch_size, 
    shuffle=False, 
    num_workers=hparams.n_workers
)

In [15]:

import os

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score



import torch


from tqdm import tqdm 
import pandas as pd



if True:

    model.eval()
    gender_pred = []
    gender_true = []

    for batch in tqdm(test_set):
        x,  y_g = batch
        x = x.to(device)
        y_hat_g = model(x)
        if y_hat_g>0.5:
            gender_pred.append(1)
        else :
            gender_pred.append(0)

        gender_true.append(y_g)

        print(accuracy_score(gender_true, gender_pred))
        print(classification_report(gender_pred,gender_true,digits=4))



Testing Model on NISP Dataset
#Cores = 2	#GPU = -1


100%|██████████| 2868/2868 [00:28<00:00, 101.84it/s]

0.9721059972105998
              precision    recall  f1-score   support

           0     0.9760    0.9708    0.9734      1508
           1     0.9678    0.9735    0.9707      1360

    accuracy                         0.9721      2868
   macro avg     0.9719    0.9722    0.9720      2868
weighted avg     0.9721    0.9721    0.9721      2868



In [16]:
torch.save(model, 'cnnlstmatt.pt')